In [1]:
from numpy import zeros, max, sqrt, isnan, isinf, dot, diag, count_nonzero
from numpy.linalg import svd, linalg, LinAlgError, norm
from scipy.linalg import svd as scipy_svd
#from scipy.sparse.linalg import svds as scipy_svds

In [2]:
from matrixSketcher import MatrixSketcher

In [137]:
class FrequentDirections(MatrixSketcher):

    def __init__(self , columns, rows):
        self.class_name = 'FrequentDirections'
        self.columns = columns
        self.rows = rows
#         self.maxRow = 2*self.rows
#         self._sketchMatrix = zeros((self.maxRow, self.columns)) 
        self._sketchMatrix = zeros((self.rows, self.columns)) 
        self.nextZeroRow = 0
                 
    def append(self,vector):     
        if count_nonzero(vector) == 0:
            return

#         if self.nextZeroRow >= self.maxRow:
#             self.__operate__()
        
        # If the approximate matrix is full, call the operate method to free half of the columns
        if self.nextZeroRow >= self.rows:
            self.__operate__()

        # Push the new vector to the next zero row and increase the next zero row index
        self._sketchMatrix[self.nextZeroRow,:] = vector 
        self.nextZeroRow += 1


    # Shrink the approximate matrix
    def __operate__(self):
        # Calculating SVD
        [_,s,Vt] = svd(self._sketchMatrix , full_matrices=False)
        #Shrink the sketch matrix
        self._sketchMatrix[:len(s),:] = dot(diag(s), Vt[:len(s),:])
        self._sketchMatrix[int(len(s)/2):,:] = 0
        self.nextZeroRow = int(len(s)/2)
        
        
#         try:
#             [_,s,Vt] = svd(self._sketchMatrix , full_matrices=False)
#         except linalg.LinAlgError as err:
#             [_,s,Vt] = scipy_svd(self._sketchMatrix, full_matrices = False)
        
        # If lenghts of matrix s is greater than number of rows
        # shrink the matrix s
#         if len(s) >= self.rows:
#             sShrunk = sqrt(s[:self.rows]**2 - s[self.rows-1]**2)
#             self._sketchMatrix[:self.rows:,:] = dot(diag(sShrunk), Vt[:self.rows,:])
#             self._sketchMatrix[self.rows:,:] = 0
#             self.nextZeroRow = self.rows
#         else:
#             self._sketchMatrix[:len(s),:] = dot(diag(s), Vt[:len(s),:])
#             self._sketchMatrix[len(s):,:] = 0
#             self.nextZeroRow = self.rows/2
#             self.nextZeroRow
        
    # Return the sketch matrix
    def get(self):
        return self._sketchMatrix[:self.rows,:]

In [138]:
import numpy as np

In [139]:
A = 100* np.random.rand(15,100)

In [140]:
A

array([[ 65.77282979,   2.4942881 ,  74.96516603, ...,  93.99723624,
         24.52915334,  72.15973051],
       [ 54.46728206,  24.14411667,   1.35794203, ...,  90.1678616 ,
         71.65331301,   7.91426698],
       [ 67.56133148,  39.49112078,  75.47707498, ...,  30.73700689,
         67.63073557,  11.05880127],
       ..., 
       [ 73.56105306,  71.66119067,  99.39929808, ...,  80.51993179,
         13.05249098,  29.34253435],
       [  3.77385362,   9.18200697,  57.54584547, ...,  46.25119315,
         19.54905651,  70.32251597],
       [ 17.66774265,  91.81194554,   8.17878825, ...,  17.25515296,
         26.21408016,  37.3865551 ]])

In [141]:
fd =  FrequentDirections(15,10)
for i in range(99):
    row = A[:,i]
    fd.append(row)

TypeError: slice indices must be integers or None or have an __index__ method

In [135]:
matrix = fd.get()

In [136]:
matrix

array([[-504.54073396, -472.06869108, -563.47238687, -539.59225463,
        -514.20958521, -510.31715969, -490.26726232, -514.01116026,
        -498.91746962, -478.20280225, -499.19657773, -473.53050583,
        -439.07362622, -481.34098829, -544.05397734],
       [ -77.36704581,   33.62294086,   59.79115188,  -52.67276619,
          85.58184495,   99.85203204,  127.9008868 ,   27.70719149,
          99.39595032,   34.70446385, -118.40194151,  -52.93935438,
         -42.35679882,  -21.38184728, -196.92748875],
       [ -13.49653345,   44.49006335,  -63.43851727,   57.09790649,
         154.58917784,  -74.60134861,   -1.4790501 ,  -97.42514345,
         -62.91130293,   61.74173428, -142.65713873,  -30.64643269,
         139.99464772,   15.66923972,   34.37811596],
       [  50.39271584,   92.80302487,  105.04145181,  -88.11508705,
        -126.50861681,  -40.03687517,  127.70470121,  -79.23976965,
         -86.23035964,    2.52136694,  -35.52820497,  -47.88437762,
          20.95908169,

In [98]:
approxCovarianceMatrix = dot(matrix.transpose(),matrix)

In [12]:
approxCovarianceMatrix

array([[ 199183.64105674,  218036.83291039,  213354.90269641,
         190126.65397378,  194968.99828445,  214661.87836235,
         214824.75494672,  193771.80047744,  221786.96248809,
         203083.88529533,  217504.50691566,  202768.52854814,
         200694.70538618,  195960.64131011,  204460.92433377],
       [ 218036.83291039,  254296.71710269,  246779.32835732,
         225592.66052403,  223162.61617308,  243728.38508159,
         248134.64249593,  225099.07307122,  250339.455056  ,
         231010.74334872,  254208.0423448 ,  233221.60790554,
         232094.33930258,  222947.98027526,  230416.27233598],
       [ 213354.90269641,  246779.32835732,  256025.16637465,
         230126.2851211 ,  222003.22628737,  252015.53845328,
         248177.03630468,  222379.96863142,  259110.17974628,
         235489.20659516,  251269.58201432,  238017.34617585,
         234922.97202712,  228116.75752146,  233232.13777866],
       [ 190126.65397378,  225592.66052403,  230126.2851211 ,
     

In [14]:
approxCovarianceMatrix2 = dot(A.transpose(),A)

In [15]:
approxCovarianceMatrix2

array([[ 52506.01106826,  31433.11928973,  41442.19418838, ...,
         45114.1594982 ,  39020.04692864,  44425.497424  ],
       [ 31433.11928973,  39262.61058979,  35889.56719816, ...,
         32677.55771559,  29527.6165797 ,  32046.75948525],
       [ 41442.19418838,  35889.56719816,  54625.86785636, ...,
         45447.67793142,  40548.99742303,  40075.95365588],
       ..., 
       [ 45114.1594982 ,  32677.55771559,  45447.67793142, ...,
         55804.34756686,  38458.87856473,  43901.96180028],
       [ 39020.04692864,  29527.6165797 ,  40548.99742303, ...,
         38458.87856473,  48340.0885645 ,  36848.02198127],
       [ 44425.497424  ,  32046.75948525,  40075.95365588, ...,
         43901.96180028,  36848.02198127,  49746.37482157]])

In [33]:
A = 100* np.random.rand(15,10)

In [34]:
A

array([[  9.13683495e+01,   4.75170797e+01,   8.48548078e+01,
          3.47901189e+01,   5.25631998e+01,   3.14644655e+01,
          2.35121828e+01,   7.86824682e+00,   1.99169407e+00,
          2.03196219e+01],
       [  4.50655695e+01,   5.93919690e+01,   5.35347504e+01,
          9.01997153e+01,   5.50237992e+01,   9.71473737e+01,
          4.39581405e+01,   2.78985522e-02,   8.25855460e+01,
          7.14304483e+01],
       [  5.90642119e+01,   1.62238953e+01,   7.15717259e+01,
          7.57489911e+01,   9.98934164e+01,   9.08033336e+00,
          8.09007869e+01,   9.37864206e+01,   1.37924050e+00,
          4.72396815e+01],
       [  8.78924874e+01,   1.42469576e+01,   4.74222598e+01,
          1.56376280e+01,   1.70825464e+01,   5.63203704e+01,
          1.21273777e+01,   9.15214127e+01,   8.67037081e+01,
          3.70796722e+01],
       [  8.41978560e+01,   8.29095040e+01,   1.05902199e+00,
          2.11608873e+01,   4.62868852e+01,   6.67870699e+01,
          7.57847070e+01

In [35]:
U, s, V = svd(A,full_matrices=True)

In [37]:
s

array([ 641.25855545,  194.89682012,  147.58406169,  122.40965389,
        116.41323723,   99.16505873,   95.77053128,   87.76287353,
         58.47661079,   42.15444507])

In [48]:
sketchMatrix = zeros((10, 10))
sketchMatrix[:len(s),:] = dot(diag(s), V[:len(s),:])
sketchMatrix

array([[ -2.40956709e+02,  -1.67137446e+02,  -2.32137238e+02,
         -2.07552446e+02,  -2.24948000e+02,  -1.89818481e+02,
         -2.08270458e+02,  -2.17604072e+02,  -1.61314526e+02,
         -1.57697836e+02],
       [  3.56778850e+01,  -6.90676138e+01,   6.64606665e+00,
         -3.39733602e+00,   4.22073801e+01,  -8.88426904e+01,
          3.24458621e+01,   9.58593647e+01,  -1.09360195e+02,
         -3.14976468e+00],
       [  9.40179537e+01,  -4.50990208e+01,  -5.91264506e+01,
          1.20424374e+01,  -6.23774475e+01,  -2.00224950e+00,
         -2.25382721e+01,   1.98253250e+01,   4.26981481e+01,
          2.54498819e+01],
       [  3.15816756e+01,   7.57857961e+01,   1.91129626e+00,
         -7.64164399e+00,  -2.44888152e+01,  -7.48875600e+01,
          2.69008803e+01,  -1.07911064e+01,   1.35889811e+01,
         -3.07988963e+01],
       [  2.55236458e+01,   7.93556831e+00,  -6.20850287e+01,
         -4.96748082e+00,   2.77059293e+01,   2.92619044e+01,
          5.97293997e+01

In [52]:
sketchMatrix[int(len(s)/2):,:] = 0
sketchMatrix

array([[-240.9567093 , -167.1374461 , -232.13723805, -207.55244605,
        -224.94800005, -189.81848119, -208.27045829, -217.60407157,
        -161.3145263 , -157.69783594],
       [  35.67788503,  -69.06761384,    6.64606665,   -3.39733602,
          42.20738015,  -88.84269044,   32.44586207,   95.85936468,
        -109.36019511,   -3.14976468],
       [  94.01795372,  -45.0990208 ,  -59.12645056,   12.04243739,
         -62.37744746,   -2.0022495 ,  -22.53827207,   19.825325  ,
          42.6981481 ,   25.44988188],
       [  31.58167556,   75.7857961 ,    1.91129626,   -7.64164399,
         -24.48881516,  -74.88755998,   26.90088026,  -10.79110637,
          13.58898113,  -30.79889635],
       [  25.52364577,    7.93556831,  -62.08502868,   -4.96748082,
          27.70592931,   29.26190444,   59.72939967,  -48.18004874,
         -37.95706433,    2.20245951],
       [   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0